In [10]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import skimage

caffe_root = !cd caffe/python && pwd
print(caffe_root)
import sys
sys.path.insert(0, caffe_root[0])  

import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


DEMO_DIR = '.'

categories = [ 'Angry' , 'Disgust' , 'Fear' , 'Happy'  , 'Neutral' ,  'Sad' , 'Surprise']

['/bin/bash: line 0: cd: caffe/python: No such file or directory']


In [0]:
def showimage(im):
    if im.ndim == 3:
        im = im[:, :, ::-1]
    plt.set_cmap('jet')
    plt.imshow(im,vmin=0, vmax=0.2)
    

def vis_square(data, padsize=1, padval=0):
    data -= data.min()
    data /= data.max()
    
    # force the number of filters to be square
    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, padsize), (0, padsize)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant', constant_values=(padval, padval))
    
    # tile the filters into an image
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3) + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    
    showimage(data)

In [7]:
  cur_net_dir = 'VGG_S_rgb'

  mean_filename=os.path.join(DEMO_DIR,cur_net_dir,'mean.binaryproto')
  proto_data = open(mean_filename, "rb").read()
  a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
  mean  = caffe.io.blobproto_to_array(a)[0]

  net_pretrained = os.path.join(DEMO_DIR,cur_net_dir,'EmotiW_VGG_S.caffemodel')
  net_model_file = os.path.join(DEMO_DIR,cur_net_dir,'deploy.prototxt')
  VGG_S_Net = caffe.Classifier(net_model_file, net_pretrained,
                        mean=mean,
                        channel_swap=(2,1,0),
                        raw_scale=255,
                        image_dims=(256, 256))

  #input_image = caffe.io.load_image(os.path.join(DEMO_DIR,cur_net_dir,'demo_image.png'))
  input_image = skimage.img_as_float(skimage.io.imread(os.path.join(DEMO_DIR,cur_net_dir,'demo_image.png'))).astype(np.float32)
  prediction = VGG_S_Net.predict([input_image],oversample=False)
  print (('predicted category is {0}').format(categories[prediction.argmax()]))

predicted category is Disgust
